In [1]:
import pandas as pd
import glob
import os

In [4]:
# Update the path to point to the folder
folder_path = '/workspaces/Futures-First/BackTest/data/LE Outrights/'

# Get the full paths of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Print the list of CSV file paths
csv_files = sorted(csv_files)

for i in csv_files:
    path = i
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    df.set_index('Date', inplace=True)
    def calculate_bollinger_bands(df, window=20, std_dev=2):
        df['SMA'] = df['Close'].rolling(window=window).mean()
        df['STD'] = df['Close'].rolling(window=window).std()
        df['Upper Band'] = df['SMA'] + (df['STD'] * std_dev)
        df['Lower Band'] = df['SMA'] - (df['STD'] * std_dev)
        return df

    def detect_outside_bands(df):
        df['Outside Upper'] = (df['Low'] > df['Upper Band']) & (df['High'] > df['Upper Band'])
        df['Outside Lower'] = (df['Low'] < df['Lower Band']) & (df['High'] < df['Lower Band'])
        return df

    df = calculate_bollinger_bands(df)
    df = detect_outside_bands(df)
    x = df['Outside Lower'].sum()
    y = df['Outside Upper'].sum()
    ls = []
    ls = [path.split('/')[-1][:-4],str(df.index[0].date()),str(df.index[-1].date()),x,y]
    new_df = pd.read_csv('/workspaces/Futures-First/BackTest/BB_count.csv')
    new_df.loc[len(new_df)] = ls
    new_df.drop_duplicates(subset='Contract', keep='first', inplace=True)
    new_df.to_csv('/workspaces/Futures-First/BackTest/BB_count.csv',index=False)

In [4]:
path = '/workspaces/Futures-First/BackTest/data/LE Outrights/LE Dec24_5min.csv'
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df.set_index('Date', inplace=True)
def calculate_bollinger_bands(df, window=20, std_dev=2):
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['STD'] = df['Close'].rolling(window=window).std()
    df['Upper Band'] = df['SMA'] + (df['STD'] * std_dev)
    df['Lower Band'] = df['SMA'] - (df['STD'] * std_dev)
    return df

def detect_outside_bands(df):
    df['Outside Upper'] = (df['Low'] > df['Upper Band']) & (df['High'] > df['Upper Band'])
    df['Outside Lower'] = (df['Low'] < df['Lower Band']) & (df['High'] < df['Lower Band'])
    return df

df = calculate_bollinger_bands(df)
df = detect_outside_bands(df)
x = df['Outside Lower'].sum()
y = df['Outside Upper'].sum()
ls = []
ls = [path.split('/')[-1][:-4],str(df.index[0].date()),str(df.index[-1].date()),x,y]
# new_df = pd.read_csv('/workspaces/Futures-First/BackTest/BB_count.csv')
# new_df.loc[len(new_df)] = ls
# new_df.drop_duplicates(subset='Contract', keep='first', inplace=True)
# new_df.to_csv('/workspaces/Futures-First/BackTest/BB_count.csv',index=False)

In [16]:
def determine_timeframe(df):
    time_diff = df.index.to_series().diff().mode().iloc[0]
    
    if time_diff == pd.Timedelta(minutes=5):
        return '5min'
    elif time_diff == pd.Timedelta(minutes=15):
        return '15min'
    elif time_diff == pd.Timedelta(minutes=30):
        return '30min'
    elif time_diff == pd.Timedelta(hours=1):
        return '1H'
    elif time_diff == pd.Timedelta(days=1):
        return '1D'
    else:
        return f'Custom ({time_diff})'

# Determine the timeframe
timeframe = determine_timeframe(df)
print(f"The timeframe of the data is: {timeframe}")

The timeframe of the data is: 5min


In [7]:
import pandas as pd

def get_continuous_candles_df(df, min_candles=1):
    results = []
    current_streak = 0
    start_date = None
    band_type = None

    for date, row in df.iterrows():
        if row['Outside Upper']:
            if current_streak == 0 or band_type != 'Upper':
                if current_streak >= min_candles:
                    results.append({
                        'Start Date': start_date,
                        'End Date': prev_date,
                        'Total Candles': current_streak,
                        'Start Price': df.loc[start_date, 'Open'],
                        'End Price': df.loc[prev_date, 'Close'],
                        'Band': band_type
                    })
                start_date = date
                current_streak = 1
                band_type = 'Upper'
            else:
                current_streak += 1
        elif row['Outside Lower']:
            if current_streak == 0 or band_type != 'Lower':
                if current_streak >= min_candles:
                    results.append({
                        'Start Date': start_date,
                        'End Date': prev_date,
                        'Total Candles': current_streak,
                        'Start Price': df.loc[start_date, 'Open'],
                        'End Price': df.loc[prev_date, 'Close'],
                        'Band': band_type
                    })
                start_date = date
                current_streak = 1
                band_type = 'Lower'
            else:
                current_streak += 1
        else:
            if current_streak >= min_candles:
                results.append({
                    'Start Date': start_date,
                    'End Date': prev_date,
                    'Total Candles': current_streak,
                    'Start Price': df.loc[start_date, 'Open'],
                    'End Price': df.loc[prev_date, 'Close'],
                    'Band': band_type
                })
            current_streak = 0
            band_type = None
        
        prev_date = date

    # Check for the last streak
    if current_streak >= min_candles:
        results.append({
            'Start Date': start_date,
            'End Date': prev_date,
            'Total Candles': current_streak,
            'Start Price': df.loc[start_date, 'Open'],
            'End Price': df.loc[prev_date, 'Close'],
            'Band': band_type
        })

    return pd.DataFrame(results)

# Usage example
min_candles = 2
continuous_candles_df = get_continuous_candles_df(df, min_candles)
print(continuous_candles_df)

           Start Date            End Date  Total Candles  Start Price  \
0 2024-06-14 13:30:00 2024-06-14 13:35:00              2      185.925   
1 2024-07-09 16:15:00 2024-07-09 16:20:00              2      185.025   
2 2024-07-11 13:30:00 2024-07-11 13:35:00              2      185.400   
3 2024-07-15 16:25:00 2024-07-15 16:30:00              2      186.250   
4 2024-07-22 13:35:00 2024-07-22 13:40:00              2      186.150   
5 2024-08-05 13:30:00 2024-08-05 13:35:00              2      178.125   
6 2024-08-20 13:55:00 2024-08-20 14:00:00              2      177.000   
7 2024-08-28 13:35:00 2024-08-28 13:40:00              2      177.775   
8 2024-08-29 17:35:00 2024-08-29 17:40:00              2      176.625   
9 2024-08-30 13:40:00 2024-08-30 13:45:00              2      177.625   

   End Price   Band  
0    186.100  Upper  
1    184.925  Lower  
2    185.325  Upper  
3    186.325  Upper  
4    186.350  Upper  
5    178.100  Lower  
6    176.650  Lower  
7    177.725  Lower 

In [25]:
def get_continuous_candles_df(df, min_candles=1):
    results = []
    current_streak = 0
    start_date = None
    band_type = None

    for date, row in df.iterrows():
        if row['Outside Upper']:
            if current_streak == 0 or band_type != 'Upper':
                if current_streak >= min_candles:
                    results.append({
                        'Start Date': start_date,
                        'End Date': prev_date,
                        'Total Candles': current_streak,
                        'Start Price': df.loc[start_date, 'Open'],
                        'End Price': df.loc[prev_date, 'Close'],
                        'Band': band_type
                    })
                start_date = date
                current_streak = 1
                band_type = 'Upper'
            else:
                current_streak += 1
        elif row['Outside Lower']:
            if current_streak == 0 or band_type != 'Lower':
                if current_streak >= min_candles:
                    results.append({
                        'Start Date': start_date,
                        'End Date': prev_date,
                        'Total Candles': current_streak,
                        'Start Price': df.loc[start_date, 'Open'],
                        'End Price': df.loc[prev_date, 'Close'],
                        'Band': band_type
                    })
                start_date = date
                current_streak = 1
                band_type = 'Lower'
            else:
                current_streak += 1
        else:
            if current_streak >= min_candles:
                results.append({
                    'Start Date': start_date,
                    'End Date': prev_date,
                    'Total Candles': current_streak,
                    'Start Price': df.loc[start_date, 'Open'],
                    'End Price': df.loc[prev_date, 'Close'],
                    'Band': band_type
                })
            current_streak = 0
            band_type = None
        
        prev_date = date

    # Check for the last streak
    if current_streak >= min_candles:
        results.append({
            'Start Date': start_date,
            'End Date': prev_date,
            'Total Candles': current_streak,
            'Start Price': df.loc[start_date, 'Open'],
            'End Price': df.loc[prev_date, 'Close'],
            'Band': band_type
        })

    return pd.DataFrame(results)


folder_path = '/workspaces/Futures-First/BackTest/data/HE Outrights/'

# Get the full paths of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Print the list of CSV file paths
csv_files = sorted(csv_files)

for i in csv_files:
    path = i
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    df.set_index('Date', inplace=True)
    def calculate_bollinger_bands(df, window=20, std_dev=3):
        df['SMA'] = df['Close'].rolling(window=window).mean()
        df['STD'] = df['Close'].rolling(window=window).std()
        df['Upper Band'] = df['SMA'] + (df['STD'] * std_dev)
        df['Lower Band'] = df['SMA'] - (df['STD'] * std_dev)
        return df

    def detect_outside_bands(df):
        df['Outside Upper'] = (df['Low'] > df['Upper Band']) & (df['High'] > df['Upper Band'])
        df['Outside Lower'] = (df['Low'] < df['Lower Band']) & (df['High'] < df['Lower Band'])
        return df

    df = calculate_bollinger_bands(df)
    df = detect_outside_bands(df)
    x = df['Outside Lower'].sum()
    y = df['Outside Upper'].sum()
    ls = []
    
    
    min_candles = 1
    continuous_candles_df = get_continuous_candles_df(df, min_candles)
    if not continuous_candles_df.empty and 'Total Candles' in continuous_candles_df.columns:
        median_value = continuous_candles_df['Total Candles'].median()
    else:
        median_value = None  

    ls = [path.split('/')[-1][:-4], str(df.index[0].date()), str(df.index[-1].date()), x, y, median_value]
    
    with open('/workspaces/Futures-First/BackTest/BB_count.csv', 'a') as f:
        # Write the list (as rows)
        list_as_row = pd.DataFrame([ls],columns=['Contract','Start Date','End Date','Lower','Upper','Median'])
        list_as_row.to_csv(f, index=False, header=True)

        # Append a blank row
        f.write("\n")

        # Write the DataFrame
        continuous_candles_df.to_csv(f, index=False)
        f.write("\n")
    
    

In [21]:
continuous_candles_df

""
